In [1]:
import heapq
import numpy as np
import random

In [29]:
def given_random(lenght, tot, std):
    result = []
    prime_mean = tot/(lenght)

    for n in range(lenght-1):
        mean = tot/(lenght-n)
        std1 = min(prime_mean+std-mean, mean-(prime_mean-std))
        if std1 < 0:
            std1 = 0
        hi_band = round(mean + std1)
        low_band = round(mean - std1)
        print(low_band, hi_band)
        result.append(random.randint(low_band, hi_band))
        tot -= result[-1]
    result.append(tot)

    return result


In [75]:
class Graph:

    def __init__(self, num_nodes, edges):
        self.num_nodes = num_nodes
        self.data = [[] for _ in range(num_nodes)]
        for n1, n2 in edges:
            self.data[n1].append(n2)
            self.data[n2].append(n1)

In [83]:
graph1 = Graph(6,[[0,1], [0,2], [0,3], [1,2], [2,4], [4,5]])

In [85]:
graph1.data

[[1, 2, 3], [0, 2], [0, 1, 4], [0], [2, 5], [4]]

In [ ]:
def bfs(graph, root):
    queue = []
    discovered = [False] * len(graph.data)

    discovered[root] = True
    queue.append(root)